# Laboratorio #1: Modelo Booleano de Recuperación de Información

Un Modelo de Recuperación de Información (MRI) es un sistema diseñado para facilitar la búsqueda y extracción de información relevante de grandes bases de datos o colecciones de documentos. Estos modelos son cruciales en áreas como los motores de búsqueda en internet, los sistemas de gestión documental y las bibliotecas digitales. Permiten a los usuarios encontrar información específica basada en criterios de búsqueda variados. Algunos de los modelos más significativos incluyen el modelo booleano, que utiliza operadores lógicos para la búsqueda; y el modelo vectorial, que emplea conceptos de álgebra lineal para relacionar documentos y consultas.

Durante el laboratorio se estará trabajando con el Modelo Booleano. Este de define como:
- **D**: Vectores binarios asociados a los términos indexados.
- **Q**: Expresión booleana sobre los términos indexados, utilizando los operadores: _and_, _or_, _not_.
- **F**: Álgebra booleana y teoría de conjuntos.
- **R**: $sim(d_j, q) = 
		 		\left\{
		 			\begin{array}{ll} 
		 				1 & si \ \exists \overrightarrow{q_{cc}} : \overrightarrow{q_{cc}} \in \overrightarrow{q_{fnd}} \ \wedge \ (\forall t_i :\  g_i(\overrightarrow{d_j}) = g_i(\overrightarrow{q_{cc}}))\\ \\
		 				0 & en \ otro \ caso
		 			\end{array} 
		 		\right.
		 	$

In [2]:
# Configurando el entorno

# Fuente del corpus
import ir_datasets

# Trabajo con los operadores lógicos
from sympy import sympify, to_dnf, Not, And, Or

# Facilita el trabajo con los términos indexados del corpus
from gensim.corpora import Dictionary

# Funciones útiles auxiliares
from teacher_help import tokenize

### Cranfield

El corpus Cranfield es un conjunto clásico de datos en el campo de la Recuperación de Información compuesto por aproximadamente 1,400 resúmenes de artículos de investigación en aerodinámica. Cada documento en este corpus incluye un título, un resumen conciso del contenido, y en algunos casos, palabras clave y referencias bibliográficas. Acompañando a estos documentos, hay alrededor de 225 consultas de prueba y sus correspondientes juicios de relevancia, proporcionados por expertos, que indican la pertinencia de cada documento para una consulta específica. Este diseño estructurado y su enfoque específico en temas de aerodinámica hacen del corpus una herramienta esencial para evaluar la eficacia de Sistemas de Recuperación de Información (SRI), sirviendo como un modelo estándar para pruebas comparativas y consistentes en esta disciplina.

La variable **dataset**, instancia de la clase **ir_datasets.datasets.base.Dataset**, tiene 3 funciones:
1. _docs_iter()_
2. _queries_iter()_
3. _qrels_iter()_

Pero durante esta clase solo se trabajará con la primera de ellas. La función **docs_iter()** devuelve un objeto iterable de tuplas de dimensión 5, referentes a los documentos. Los campos de cada tupla son:
  - Identificador (str)
  - Título (str)
  - Texto (str)
  - Autor (str)
  - Referencia bibliográfica (str)


### Ejercicio #1: Cargue en memoria el corpus Cranfield y tokenice cada documentos.
###### Ayuda: Consulte la función `tokenize/1` hecha por los profesores.

In [3]:
corpus = ir_datasets.load('cranfield')
tokenized_docs = []
for doc in corpus.docs_iter()[:10]:
    tokenized_docs.append((doc[0],tokenize(doc[2])))
print(tokenized_docs)

[('1', ['experimental', 'investigation', 'of', 'the', 'aerodynamic', 'of', 'a', 'wing', 'in', 'a', 'slipstream', 'an', 'experimental', 'study', 'of', 'a', 'wing', 'in', 'a', 'propeller', 'slipstream', 'be', 'make', 'in', 'order', 'to', 'determine', 'the', 'spanwise', 'distribution', 'of', 'the', 'lift', 'increase', 'due', 'to', 'slipstream', 'at', 'different', 'angle', 'of', 'attack', 'of', 'the', 'wing', 'and', 'at', 'different', 'free', 'stream', 'to', 'slipstream', 'velocity', 'ratio', 'the', 'result', 'be', 'intend', 'in', 'part', 'as', 'an', 'evaluation', 'basis', 'for', 'different', 'theoretical', 'treatment', 'of', 'this', 'problem', 'the', 'comparative', 'span', 'loading', 'curve', 'together', 'with', 'support', 'evidence', 'show', 'that', 'a', 'substantial', 'part', 'of', 'the', 'lift', 'increment', 'produce', 'by', 'the', 'slipstream', 'be', 'due', 'to', 'a', 'or', 'boundary', 'layer', 'control', 'effect', 'the', 'integrate', 'remain', 'lift', 'increment', 'after', 'subtract'

### Creando mi primera estructura de datos para un MRI

Los términos invertidos, también conocidos como **índices invertidos**, son estructuras de datos utilizadas en Sistemas de Recuperación de Información (SRI) para asociar términos (o palabras) con los documentos que los contienen. Básicamente, son diccionarios que asignan términos a una lista de documentos en los que aparecen esos términos. Esto permite una búsqueda eficiente de documentos que contienen ciertos términos.
 

### Ejercicio #2: Implemente una función que dado un corpus tokenizado, devuelva los índices invertidos.
Considere que cada documento posee un identificador único.

In [4]:
def build_inverted_index(data):
    """
    Build an inverted index structure
    
    Args:
    - data : [(str, [str])]
        List of tuples of size 2. The first position contains the document identifier and the second the list of document tokens.

    Return:
    - dict
    
    """
    
    inv_ind = {}
    
    for element in data:
        ind = element[0]
        tokens = element[1]
        
        for token in tokens:
            if token not in inv_ind:
                inv_ind[token] = {ind}
            inv_ind[token].add(ind)
    
    return inv_ind
            

print(build_inverted_index(tokenized_docs))

{'experimental': {'1'}, 'investigation': {'9', '1', '8'}, 'of': {'2', '6', '5', '7', '10', '9', '1', '4', '8'}, 'the': {'2', '6', '5', '7', '10', '9', '1', '3', '4', '8'}, 'aerodynamic': {'1', '5'}, 'a': {'2', '6', '5', '7', '10', '9', '1', '3', '4', '8'}, 'wing': {'1'}, 'in': {'2', '6', '5', '7', '9', '1', '3', '4', '8'}, 'slipstream': {'1'}, 'an': {'2', '10', '9', '1', '8'}, 'study': {'9', '2', '1', '8'}, 'propeller': {'1'}, 'be': {'2', '6', '5', '7', '10', '9', '1', '3', '4', '8'}, 'make': {'10', '4', '9', '1'}, 'order': {'1'}, 'to': {'2', '6', '5', '7', '9', '1', '4', '8'}, 'determine': {'1'}, 'spanwise': {'1'}, 'distribution': {'4', '1'}, 'lift': {'1'}, 'increase': {'1', '7'}, 'due': {'1'}, 'at': {'5', '7', '10', '9', '1', '6', '8'}, 'different': {'2', '1'}, 'angle': {'9', '1'}, 'attack': {'1'}, 'and': {'2', '6', '7', '10', '9', '1', '4', '8'}, 'free': {'10', '2', '1'}, 'stream': {'10', '2', '1'}, 'velocity': {'4', '1', '7'}, 'ratio': {'1'}, 'result': {'7', '10', '9', '1', '8'}, '

### Representación de la consulta en FND

La Forma Normal Disyuntiva es una forma estándar de expresar una fórmula lógica booleana como una disyunción de cláusulas, donde cada cláusula es una conjunción de literales (variables booleanas o sus negaciones). Esta representación es útil en la lógica booleana y en la teoría de la computación porque permite realizar operaciones y simplificaciones sobre las expresiones booleanas de manera sistemática. Además, muchas funciones lógicas pueden ser representadas eficientemente en DNF.

### Ejercicio #3: Implemente una función que dada una query booleana, devuelva la FND de la misma.
###### Ayuda #1: Considere usar las funciones `sympify/1` y `to_dnf/2`.
###### Ayuda #2: Considere usar los símbolos `&`, `|` y `~` para sustituir los operadores lógicos `and`, `or` y `not`.

In [35]:
def query_to_dnf(query):
    """
    Transforms a boolean expression into its disjunctive normal form
    
    Args:
    - query : str
        Boolean query.

    Return:
    - instance of class Or
    
    """
    consulta = tokenize(query)
    for i,token in enumerate(consulta):
        if token == 'and':
            consulta[i] = '&'
        elif token == 'or':
            consulta[i] = '|'
        elif token == 'not':
            consulta[i] = '~'

    consulta = ' '.join(consulta)  
    consulta = sympify(consulta)
    
    return to_dnf(consulta)

query = "A OR B"
query_dnf = query_to_dnf(query)
print(query_dnf)
type(query_dnf)
# query_dnf

a | b


Or

### Ejercicio #4: Implemente la función de similitud del modelo booleano, de forma que dada la consulta y el conjunto de documentos tokenizados la función devuelva aquellos documentos que satifacen a la consulta.
###### Ayuda: Considere usar las funciones `Dictionary/1`, `Dictionary.token2id/0` y `Dictionary.doc2bow/1`.

In [34]:
def get_matching_docs(query_dnf, tokenized_docs_no_ind):
    """
    Similarity function

    Args:
        - query_dnf : Instance of class Or
        Query in FND

    Return:
    - [boolean]. each position matches a document in the corpus. If the value is True then the document satisfies the query, False otherwise.

    """
    
    vector = [False for _ in range(len(tokenized_docs_no_ind))]
    
    dct = Dictionary(tokenized_docs_no_ind)
    corpus = [dct.doc2bow(doc) for doc in tokenized_docs_no_ind]
    
    for doc in corpus:
        flag = False
        for clause in query_dnf.args:
            cur = True
            for term in clause.args:
                term = str(term)
                
                if term[0] == '~':
                    if dct.token2id.get(term[1:])  in dict(doc):
                        cur = False
                else:
                    if dct.token2id.get(term) not in dict(doc):
                        cur = False
                    
                #print(corpus.index(doc), clause, cur)
                flag |= cur
        
        vector[corpus.index(doc)] = flag
    return vector

consulta  = "investigation AND (of OR NOT experimental)"
consulta_dnf = query_to_dnf(consulta)
print(get_matching_docs(consulta_dnf, [doc for _, doc in tokenized_docs]))

[True, False, False, False, False, False, False, True, True, False]


### Ejercicio #5: ¿Cómo pudiese implementarse la función de similitud usando solamente la consulta en su FND y la estructura de los índices invertidos creada a partir del corpus procesado?